In [ ]:
import os
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error, roc_auc_score
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from joblib import dump, load
import xgboost as xgb


import warnings
warnings.filterwarnings('ignore')

In [ ]:
PATH = '../input/tabular-playground-series-nov-2021'
train = pd.read_csv(os.path.join(PATH, 'train.csv'))
test = pd.read_csv(os.path.join(PATH, 'test.csv'))
sub = pd.read_csv(os.path.join(PATH, 'sample_submission.csv'))

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train['target'].value_counts(normalize=True)

In [ ]:
def random_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

In [ ]:
columns = [col for col in train.columns if col not in ['id', 'target'] ]

In [ ]:
xgb_params= {
        "n_estimators": 20000,
        "max_depth": 10,
        "objective":"binary:logistic",
        "n_jobs": 4,
        "seed": 2001,
        'tree_method': "gpu_hist",
        "gpu_id": 0,
        "eval_metric": "auc", 
        "subsample": 0.7,
        "colsample_bytree": 0.7,
        "learning_rate": 0.01
    }

In [ ]:
SEED = 2001
N_FOLDS =10

random_seed(SEED)

auc_score = []

train = train.sample(frac=1).reset_index(drop=True)

targets = train['target'].values

kf = StratifiedKFold(n_splits = N_FOLDS, shuffle=True, random_state=SEED)    
        
oof = np.zeros((train.shape[0],))
test_preds = 0

for f, (train_idx, val_idx) in tqdm(enumerate(kf.split(train, targets))):
        df_train, df_val = train.iloc[train_idx][columns], train.iloc[val_idx][columns]
        train_target, val_target = targets[train_idx], targets[val_idx]
        
        model = xgb.XGBClassifier(**xgb_params)
        
        model.fit(
            df_train[columns], 
            train_target,
            eval_set=[(df_val[columns], val_target)],
            early_stopping_rounds=200,
            verbose=1000
        )
        
        oof_tmp = model.predict_proba(df_val[columns])[:,1]
        test_tmp = model.predict_proba(test[columns])[:,1]   
        
        oof[val_idx] = oof_tmp
        test_preds += test_tmp/N_FOLDS
        auc = roc_auc_score(val_target, oof_tmp)
        auc_score.append(auc)
        print(f'FOLD: {f} AUC: {auc} Mean AUC: {np.mean(auc_score)}')

In [ ]:
sub['target'] = test_preds
sub.to_csv('submission.csv', index=False)